<a href="https://colab.research.google.com/github/Phillistan16/imoco_recon/blob/colab_demo/colab-imoco-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# iterative Motion Compensation (iMoCo) Reconstruction Demo
This CoLab notebook is a demo of the iMoCo method
https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.27998
https://github.com/PulmonaryMRI/imoco_recon


# Setup

<!-- By default, this colab notebook should have a GPU runtime environment. To check this, go to **Runtime > Change runtime type** and see if **GPU** is selected as **Hardware accelerator**. -->

To setup, we will install [SigPy](http://sigpy.readthedocs.io), [ANTsPy](https://github.com/ANTsX/ANTsPy), and clone the iMoCo github repo.

In [ ]:
# install dependent packages sigpy, h5py, antspy
!pip install sigpy h5py
!pip install antspyx

# clone imoco recon repo
! git clone https://github.com/PulmonaryMRI/imoco_recon

# Download dataset

Now, let's download a sample Lung UTE dataset. The dataset is hosted on Zenodo: https://zenodo.org/record/3733776#.XoJlNC2ZPOQ.

<!-- The `download_lung_dataset()` function will download the datasets if it cannot find them under `data/lung/`. It takes around 8 minutes to download the datasets. -->

In [ ]:
!mkdir lung_mri
!pip install zenodo-get
!zenodo_get -d 10.5281/zenodo.3733775 -o ./lung_mri

# Crop Data (Optional)
decrease number of frequency encoding points to adapt for Colab RAM

In [13]:
import imoco_recon.imoco_py.sigpy_e.cfl as cfl
import numpy as np
num_ro = 100

datam = cfl.read_cfl('./lung_mri/MRI_Raw_datam')
datam = datam[:,:,:,:,:num_ro,:]
cfl.write_cfl('./lung_mri/MRI_Raw_datam', datam)

In [14]:
dcf2m = cfl.read_cfl('./lung_mri/MRI_Raw_dcf2m')
dcf2m = dcf2m[:,:,:,:,:num_ro,:]
cfl.write_cfl('./lung_mri/MRI_Raw_dcf2m', dcf2m)

In [15]:
trajm = cfl.read_cfl('./lung_mri/MRI_Raw_trajm')
trajm = trajm[:,:,:,:,:num_ro,:]
cfl.write_cfl('./lung_mri/MRI_Raw_trajm', trajm)

In [19]:
# release RAM
del datam
del dcf2m
del trajm

# iMoCo Reconstruction
step 1. XD-GRASP, motion-resolved reconstruction \
step 2. iMoCo reconstruction

In [16]:
!python imoco_recon/imoco_py/recon_xdgrasp.py ./lung_mri/MRI_Raw --device 0

JsenseRecon: 100% 10/10 [00:09<00:00,  1.07it/s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/sigpy/linop.py", line 95, in apply
    output = self._apply(input)
  File "/usr/local/lib/python3.6/dist-packages/sigpy/linop.py", line 1429, in _apply
    oversamp=self.oversamp, width=self.width)
  File "/usr/local/lib/python3.6/dist-packages/sigpy/fourier.py", line 129, in nufft
    output, coord, kernel='kaiser_bessel', width=width, param=beta)
  File "/usr/local/lib/python3.6/dist-packages/sigpy/interp.py", line 93, in interpolate
    input, coord, width, param, output, size=npts)
  File "cupy/core/_kernel.pyx", line 605, in cupy.core._kernel.ElementwiseKernel.__call__
  File "cupy/core/_kernel.pyx", line 90, in cupy.core._kernel._preprocess_args
TypeError: Unsupported type <class 'numpy.ndarray'>

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/sigp

In [ ]:
!python imoco_recon/imoco_py/recon_imoco.py ./lung_mri/MRI_Raw --device 0

# Display Image


In [ ]:
import sigpy.plot as pl
